In [1]:
#import important libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, accuracy_score
import math
import warnings
warnings.filterwarnings('ignore')

In [3]:
#read dataset csv file.
data = pd.read_csv("bank.csv", delimiter=";")
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [4]:
# find the features and observation in the data set.
data.shape

(4521, 17)

In [5]:
# find the datatypes of the features and target variable. 
data.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

In [7]:
bank_data = pd.get_dummies(data, columns=['job', 'marital', 'education', 'default', 'housing', 'loan',
                                                   'contact', 'month', 'poutcome'])

bank_data.head()

,age,balance,day,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,30,1787,19,79,1,-1,0,no,0,0,...,0,0,0,0,1,0,0,0,0,1
1,33,4789,11,220,1,339,4,no,0,0,...,0,0,1,0,0,0,1,0,0,0
2,35,1350,16,185,1,330,1,no,0,0,...,0,0,0,0,0,0,1,0,0,0
3,30,1476,3,199,4,-1,0,no,0,0,...,1,0,0,0,0,0,0,0,0,1
4,59,0,5,226,1,-1,0,no,0,1,...,0,0,1,0,0,0,0,0,0,1


In [8]:
bank_data.y.replace(('yes', 'no'), (1, 0), inplace=True)

In [9]:
y = pd.DataFrame(bank_data['y'])
x = bank_data.drop(['y'], axis=1)

In [10]:
x.shape, y.shape

((4521, 51), (4521, 1))

In [17]:
#Creating class for simple neural network
class MultiNeuralNetwork:

    def __init__(self):
        # Learning Rate
        self.l_rate = 0.001
        # Total iterations
        self.echops = 2000

    #Defing a function to train our data
    def train(self, x_train_data, y_train_data):

        layers = [x_train_data.shape[1], 50, 44, 65, 43, 1]
        z = 0
        weight = []
        baised = []
        a = [0] * len(layers)
        z = [0] * len(layers)
        dg = [0] * len(layers)
        da = [0] * len(layers)
        dz = [0] * len(layers)
        db = [0] * len(layers)
        dw = [0] * len(layers)
        
        a[0] = x_train_data.T 
        m = x_train_data.shape[0]
        n = x_train_data.shape[1]

        for j in range(1, len(layers)):
            
            weight.append(np.random.rand(layers[j], layers[j-1]) * 0.001 ) # 5x12288
            baised.append(np.zeros(((layers[j], 1)) ))# 5x1
 
        for k in range(self.echops):

            for i in range(len(layers) - 1):  
        
                z[i] = np.dot(weight[i], a[i]) + baised[i] # 5x209
                a[i+1] = 1 / (1 + np.exp(-z[i])) # 5x209

                # d_g = a * (1 - a)
                dg[i] = (1 / (1 + np.exp(-z[i]))) * (1 - (1 / (1 + np.exp(-z[i])))) # 5x209

                  # Loss Funtion
                # da = -(y/a) + ((1-y)/(1-a))
                da[i] = (-(y_train_data.T / a[i+1]) + ((1 - y_train_data.T) / (1 - a[i+1]))) # 5x209

                # dz = da * d_g
                dz[i] = (da[i] * dg[i]) # 5x209

                #dw = (dz * a) / m
                dw[i] = np.dot(dz[i], a[i].T)# 5x12288

                # db = np.sum(dz)
                db[i] = np.sum(dz[i], axis=1, keepdims=True) / m # 209x1

                # w = w - alpha * dw
                # b = b - alpha * db
                weight[i] =  weight[i] - np.dot(self.l_rate, dw[i]) 
                baised[i] = baised[i] - np.dot(self.l_rate, db[i]) 

        return [weight, baised]
    
    def predict(self, x_test_data, parameters):

        a = [0] * 6
        a[0] = x_test_data.T
        z = [0] * 6
        a = [0] * 6
        for i in range(4):  
            z[i] = np.dot(parameters[0][i], a[i]) + parameters[1][i]
            a[i+1] = 1 / (1 + np.exp(-z[i])) 
        return a[-1]
        return y_predict

    def accuracy(self, y_data_test, y_pred_test):

        y_pred_test = np.nan_to_num(y_pred_test)
    
        test_accuracy = 100 - (np.mean(np.abs(y_pred_test - y_data_test)) * 100)        
        return test_accuracy


def main():

    #creating object for class
    mnn = MultiNeuralNetwork()
    
    #setting data size for test and train
    train_size = int(np.ceil((len(y) * 0.80)))
    
    #Training data
    x_train_data = np.array(x[:train_size])
    y_train_data = np.array(y[:train_size])
    
    #Testing data
    x_test_data = np.array(x[train_size:])
    y_test_data = np.array(y[train_size:])
    
    
    parameters = mnn.train(x_train_data, y_train_data)
    
    y_prediction = mnn.predict(x_test_data, parameters)
    
    y_prediction_train = mnn.predict(x_train_data, parameters)
    
    accuracy = mnn.accuracy(y_test_data, y_prediction)
    accuracy_train = mnn.accuracy(y_train_data, y_prediction_train)

    print("Train accuracy: ", accuracy_train)
    print("Test accuracy: ", accuracy)


if __name__ == '__main__':
    main()

Train accuracy:  88.66463920376002
Test accuracy:  87.72123893805309


In [18]:
#comparing data accuracy with library 
# Using sklearn linear regression model
x_data_train, x_data_test, y_data_train, y_data_test = train_test_split(x, y, test_size=0.3, random_state=0)
model = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
reg = model.fit(x_data_train,y_data_train)
reg.score(x_data_test, y_data_test)
y_pred =  reg.predict(x_data_test)
print("Train accuracy", accuracy_score(y_data_train, y_data_train) * 100)
print("Test accuracy", accuracy_score(y_data_test, y_pred) * 100)

Train accuracy 100.0
Test accuracy 87.69344141488577
